# Clustering Analysis for Survey Data

This notebook performs clustering analysis on survey data using UMAP for dimensionality reduction and K-Means for clustering. The analysis includes evaluation metrics and visualizations for both 2 and 3 cluster solutions.

## 1. Data Loading and Preprocessing

- Import required libraries
- Load survey data
- Create dummy variables
- Clean column names

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import pyreadstat
import matplotlib.pyplot as plt
import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Set global plotting parameters
plt.style.use('seaborn')
np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['font.size'] = 12

# Load data
data = pd.read_stata("../Data/V1_qualflags_analysis2_ML.dta")
df, meta = pyreadstat.read_dta("../Data/V1_qualflags_analysis2_ML.dta")
variable_labels = dict(zip(data.columns, meta.column_labels))

# Create dummy variables and clean column names
data_dummies = pd.get_dummies(data)
new_cols = [col.replace('>', 'greater').replace('<', 'less').replace(',', '_') 
            for col in data_dummies.columns]
data_dummies.columns = new_cols

ModuleNotFoundError: No module named 'umap.umap_'

## 2. Dimensionality Reduction and Clustering

- Perform UMAP dimensionality reduction
- Evaluate optimal number of clusters
- Apply K-Means clustering
- Visualize results

In [ ]:
# Perform UMAP dimensionality reduction
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(data_dummies.dropna())

# Evaluate optimal number of clusters
wcss = []
silhouette_scores = []

for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embedding)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(embedding, labels))

# Plot evaluation metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
ax1.plot(range(2, 11), wcss, marker='o', linestyle='--')
ax1.set_title('Elbow Method')
ax1.set_xlabel('Number of Clusters')
ax1.set_ylabel('WCSS')

# Silhouette plot
ax2.plot(range(2, 11), silhouette_scores, marker='o', linestyle='--')
ax2.set_title('Silhouette Score')
ax2.set_xlabel('Number of Clusters')
ax2.set_ylabel('Silhouette Score')

plt.tight_layout()
plt.show()

## 3. Cluster Analysis and Export

- Generate cluster visualizations
- Save clustering results
- Export final datasets

In [ ]:
# Perform clustering for both 2 and 3 clusters
for n_clusters in [2, 3]:
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embedding)
    
    # Visualize clusters
    plt.figure(figsize=(10, 7))
    plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='viridis')
    plt.colorbar(label='Cluster')
    plt.title(f'UMAP Projection with {n_clusters} K-Means Clusters', fontsize=15)
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.show()
    
    # Save results
    result_df = data_dummies.dropna().copy()
    result_df['KMeans_Cluster'] = labels
    result_df.to_csv(f'clusters_{n_clusters}_construction.csv', index=False)